# Group spectral-peak detections on all geophone stations by time and frequency

In [ ]:
# Imports
from os.path import join

from utils_basic import SPECTROGRAM_DIR as indir, GEO_STATIONS as stations
from utils_spec import assemble_spec_filename, read_geo_spectrograms, find_geo_station_spectral_peaks
from utils_plot import plot_geo_total_psd_and_peaks, save_figure